In [105]:
import numpy as np
import csv
import pandas as pd

np.random.seed(13)  # for reproducibility
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, normalization
from keras.layers import Convolution2D, MaxPooling2D  
from keras import optimizers
from keras.optimizers import RMSprop
from keras.optimizers import SGD
from keras.optimizers import SGD

from keras.utils import np_utils
from keras import regularizers

from sklearn.model_selection import train_test_split, KFold, cross_val_score
import matplotlib.pyplot as plt

from sklearn.neural_network import MLPClassifier

In [106]:

train_labeled = pd.read_hdf("train_labeled.h5", "train")
train_unlabeled = pd.read_hdf("train_unlabeled.h5", "train")
test = pd.read_hdf("test.h5", "test")

sample_dat = np.loadtxt(open("sample.csv","rb"), delimiter = ",", skiprows = 1)

In [111]:
ID = sample_dat[:,0];
y_train_original = train_labeled.values[:,0];
X_train_original = train_labeled.values[:,1:129];
#print(X)
X_unlabeled = train_unlabeled.values[:,0:128]

X_valid = test.values[:,0:128]

dummy_y = np_utils.to_categorical(y_train_original)


In [115]:
model = Sequential()
model.name = 'NN'
#model.add(Dropout(0.2, input_shape=(100,) ))
model.add(normalization.BatchNormalization(input_shape=(128,)))
#model.add(Dense(200, input_dim = 128, activation = 'relu'))
#model.add(Dense(500, input_dim = 100, activation = 'relu'))
#model.add(Dropout(0.2))
model.add(Dense(500, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(500, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(500, activation = 'relu'))
model.add(Dropout(0.5))
#model.add(Dense(500, activation = 'relu',kernel_regularizer=regularizers.l2(0.0002)))
#model.add(Dropout(0.2))
#model.add(Dense(500, activation = 'relu',kernel_regularizer=regularizers.l2(0.0002)))
model.add(Dense(10, activation = 'softmax'))


In [116]:
X_train, X_test, Y_train, Y_test = train_test_split(X_train_original, dummy_y, test_size=0.1, random_state=1)

sgd = SGD(lr=0.01, momentum=0.9, decay=0.01, nesterov=True)
rmsprop = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
#adam = optimizers.Adam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

model.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

train

In [114]:
model.fit(X_train, Y_train, epochs = 50, verbose = 1, batch_size = 50, validation_data = (X_test, Y_test))

Train on 8100 samples, validate on 900 samples
Epoch 1/50
8100/8100 [==============================] - 3s 391us/step - loss: 1.0648 - acc: 0.6788 - val_loss: 0.5177 - val_acc: 0.8400
Epoch 2/50
3150/8100 [==========>...................] - ETA: 1s - loss: 0.7046 - acc: 0.7873

KeyboardInterrupt: 

In [117]:
model.fit(X_train_original, dummy_y, epochs = 50, verbose = 1, batch_size = 50)

Epoch 1/50
9000/9000 [==============================] - 3s 366us/step - loss: 1.0817 - acc: 0.6751
Epoch 2/50
9000/9000 [==============================] - 2s 256us/step - loss: 0.6914 - acc: 0.7958
Epoch 3/50
9000/9000 [==============================] - 2s 257us/step - loss: 0.6131 - acc: 0.8222
Epoch 4/50
9000/9000 [==============================] - 3s 279us/step - loss: 0.5405 - acc: 0.8420
Epoch 5/50
9000/9000 [==============================] - 2s 254us/step - loss: 0.4718 - acc: 0.8631
Epoch 6/50
9000/9000 [==============================] - 2s 254us/step - loss: 0.4547 - acc: 0.8628
Epoch 7/50
9000/9000 [==============================] - 2s 255us/step - loss: 0.4233 - acc: 0.8732
Epoch 8/50
9000/9000 [==============================] - 2s 259us/step - loss: 0.4014 - acc: 0.8747
Epoch 9/50
9000/9000 [==============================] - 2s 256us/step - loss: 0.3880 - acc: 0.8797
Epoch 10/50
9000/9000 [==============================] - 2s 259us/step - loss: 0.3757 - acc: 0.8881
Epoch 11/

In [122]:
y_unlabeled = model.predict_classes(X_unlabeled)
X_total = np.concatenate((X_train_original,X_unlabeled))
y_total = np.concatenate((y_train_original,y_unlabeled))
dummy_y_total = np_utils.to_categorical(y_total)

In [104]:
X_train, X_test, Y_train, Y_test = train_test_split(X_total, dummy_y_total, test_size=0.1, random_state=1)
model.fit(X_train, Y_train, epochs = 30, verbose = 1, batch_size = 50, validation_data = (X_test, Y_test))

Train on 27000 samples, validate on 3000 samples
Epoch 1/30
27000/27000 [==============================] - 7s 275us/step - loss: 0.2962 - acc: 0.9079 - val_loss: 0.1488 - val_acc: 0.9513
Epoch 2/30
27000/27000 [==============================] - 7s 252us/step - loss: 0.2856 - acc: 0.9144 - val_loss: 0.1385 - val_acc: 0.9547
Epoch 3/30
27000/27000 [==============================] - 7s 254us/step - loss: 0.2627 - acc: 0.9193 - val_loss: 0.1355 - val_acc: 0.9547
Epoch 4/30
27000/27000 [==============================] - 7s 255us/step - loss: 0.2665 - acc: 0.9174 - val_loss: 0.1404 - val_acc: 0.9537
Epoch 5/30
11850/27000 [============>.................] - ETA: 3s - loss: 0.2537 - acc: 0.9242

KeyboardInterrupt: 

In [123]:
model.fit(X_total, dummy_y_total, epochs = 30, verbose = 1, batch_size = 50)

Epoch 1/30
30000/30000 [==============================] - 7s 243us/step - loss: 0.2304 - acc: 0.9351
Epoch 2/30
30000/30000 [==============================] - 7s 242us/step - loss: 0.2336 - acc: 0.9343
Epoch 3/30
30000/30000 [==============================] - 7s 242us/step - loss: 0.2300 - acc: 0.9358
Epoch 4/30
30000/30000 [==============================] - 7s 243us/step - loss: 0.2229 - acc: 0.9360
Epoch 5/30
30000/30000 [==============================] - 7s 245us/step - loss: 0.2264 - acc: 0.9367
Epoch 6/30
30000/30000 [==============================] - 7s 246us/step - loss: 0.2161 - acc: 0.9379
Epoch 7/30
30000/30000 [==============================] - 7s 248us/step - loss: 0.2170 - acc: 0.9392
Epoch 8/30
30000/30000 [==============================] - 7s 248us/step - loss: 0.2386 - acc: 0.9356
Epoch 9/30
30000/30000 [==============================] - 7s 249us/step - loss: 0.2248 - acc: 0.9359
Epoch 10/30
30000/30000 [==============================] - 7s 246us/step - loss: 0.2172 - a

predict

In [124]:
y_valid = model.predict_classes(X_valid)

In [125]:
print(y_valid)

[7 6 3 ... 6 2 3]


In [126]:
dataframe = pd.DataFrame({'Id':ID,'y':y_valid })

dataframe.to_csv("result.csv",index=False,sep=',')